In [ ]:
#!/usr/bin/env python
import ROOT, sys, os
from ROOT import std

from larlite import larlite
from larcv import larcv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.path as path
colormap=['blue','red','magenta','green','orange','yellow','pink']
nc=len(colormap)
from ROOT import std
from ROOT import geo2d
from ROOT import cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

In [ ]:
proc = larcv.ProcessDriver("ProcessDriver")
proc.configure('unit.fcl')
flist=std.vector('string')()
#flist.push_back('out_0000_0099.root')
flist.push_back('numu_ccqe_p00_p07.root')
proc.override_input_file(flist)
proc.initialize()

larbysimg = proc.process_ptr(0)

xranges=((450,700),(450,700),(450,700))
yranges=((875,1125),(700,950),(1350,1600))

force_range = False

In [ ]:
proc.batch_process(737,1)
#proc.batch_process(726,1)
#proc.batch_process(373,1)
#proc.batch_process(243,1)
#proc.batch_process(131,1)
#proc.batch_process(14,1)

In [ ]:
pygeo   = geo2d.PyDraw()

mgr=larbysimg.Manager()
img_v = []

plane=0
for mat in mgr.InputImages():
    #img_v.append(pygeo.image(mat).transpose())
    img_v.append(pygeo.image(mat))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        plt.xlabel('Time [6 ticks]',fontsize=20)
        plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()
    plane+=1

dm=mgr.DataManager()

In [ ]:
xranges=((450,700),(450,700),(450,700))
yranges=((875,1125),(700,950),(1350,1600))

for plane in xrange(3):
    print 'Plane',plane
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')

    #Get the image
    shape_img = img_v[plane]
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)

    data=dm.Data(0)

    all_p0=mgr.Clusters(plane,0)

    for ctor in all_p0:
        pts=np.array([[pt.x,pt.y] for pt in ctor._contour])
        plt.plot(pts[:,0],pts[:,1],'-o')

    ax.set_aspect(1.0)
    plt.tight_layout()
    #ax.set_title("Plane {}".format(plane),fontweight='bold',fontsize=40)
    
    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')

    #Get the image
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)

    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    data=dm.Data(1)
    
    compound_v = data._raw_cluster_vv[plane]
    
    for compound in compound_v.get_cluster():
        for atom in compound.get_atoms():
            pts=[]
            for pt_idx in xrange(atom.size()):
                pts.append([atom.at(pt_idx).x,atom.at(pt_idx).y])
            pts = np.array(pts)
            plt.plot(pts[:,0],pts[:,1],'-o')
        for defect in compound.get_defects():
            ax.plot([0,900],[defect._split_line.y(0),defect._split_line.y(900)])

    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')

    #Get the image
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)

    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    data=dm.Data(2)
    
    for line_idx in data.index_plane(plane):
        line = data.line(line_idx)
        ax.plot([0,900],[line.y(0),line.y(900)])

    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')

    #Get the image
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)

    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    data=dm.Data(1)
    
    compound_v = data._raw_cluster_vv[plane]
    
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    data=dm.Data(2)
    
    for pt in data.points(plane):
        ax.plot([pt.x],[pt.y],marker='o',color='blue',markersize=10)
    
    for compound in compound_v.get_cluster():
        for defect in compound.get_defects():
            pt = defect._pt_defect
            ax.plot([pt.x],[pt.y],marker='o',color='red',markersize=10)

    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.patches as patches
alg=mgr.GetClusterAlg(3).Algo()

dm=mgr.DataManager()
ref_data = dm.Data(3)
tickscore0_y=[]
tickscore0_x=[]

score0_v = alg.TimeBinnedScore0Mean()
for idx in xrange(score0_v.size()):
    v = score0_v[idx]
    tickscore0_y.append(v)
    tickscore0_x.append(idx*1 + alg.TimeBinMin())
        
tickscore1_y=[]
tickscore1_x=[]
score1_v = alg.TimeBinnedScore1Mean()
for idx in xrange(score1_v.size()):
    v = score1_v[idx]
    tickscore1_y.append(v)
    tickscore1_x.append(idx*1 + alg.TimeBinMin())

tickscore0_x = np.array(tickscore0_x)
tickscore0_y = np.array(tickscore0_y)
tickscore1_x = np.array(tickscore1_x)
tickscore1_y = np.array(tickscore1_y)
ymin = tickscore0_y.min()
ymax = tickscore0_y.max()
if ymin > tickscore1_y.min(): ymin = tickscore1_y.min()
if ymax < tickscore1_y.max(): ymax = tickscore1_y.max()
    
fig,ax = plt.subplots(figsize=(16,8),facecolor='w')
plt.plot(tickscore0_x,tickscore0_y,marker='o',linestyle='-',color='red',markersize=10)
plt.plot(tickscore1_x,tickscore1_y,marker='o',linestyle='--',
         markeredgewidth=1,markeredgecolor='blue',markerfacecolor='None',markersize=10)

#minimum1_v = ref_data._time_binned_score1_minidx_v
#for idx in xrange(minimum1_v.size()):
#    xval = tickscore1_x[minimum1_v[idx]]
#    plt.plot([xval,xval],[ymin,ymax],marker='',linestyle='-',color='blue')

#minimum0_v = ref_data._time_binned_score0_minidx_v
#for idx in xrange(minimum0_v.size()):
#    xval = tickscore0_x[minimum0_v[idx]]
#    plt.plot([xval,xval],[ymin,ymax],marker='',linestyle='--',color='red')

minimum_v  = alg.TimeBinnedScoreMinIndex()
minrange_v = alg.TimeBinnedScoreMinRange()
for idx in xrange(minimum_v.size()):
    xval = tickscore0_x[minimum_v[idx]]
    plt.plot([xval,xval],[0,360],marker='',linestyle='--',color='black',linewidth=2)
    xstart = tickscore0_x[minrange_v[idx].first]
    xend   = tickscore0_x[minrange_v[idx].second]
    ax.axvspan(xstart,xend, alpha=0.3, color='orange')

plt.xlabel('Time [6 ticks]',fontsize=20,fontweight='bold')
plt.ylabel('Summed Angle Difference [deg.]',fontsize=20,fontweight='bold')
plt.tick_params(labelsize=20)
plt.tight_layout()
ax=plt.gca()
ax.set_ylim(-1,ymax*1.1)
ax.set_xlim(tickscore0_x.min(),tickscore0_x.max())
plt.grid()
#ax.set_xlim(450,480)
plt.show()

In [ ]:
import matplotlib.patches as patches
alg=mgr.GetClusterAlg(3).Algo()

dm=mgr.DataManager()
ref_data = dm.Data(3)
for plane in xrange(3):
    wirescore_y = []
    wirescore_x = []
    score_v = alg.WireBinnedScoreMean(plane)
    for idx in xrange(score_v.size()):
        wirescore_y.append(score_v[idx])
        wirescore_x.append(idx + alg.WireBinMin(plane))

    wirescore_y = np.array(wirescore_y)
    wirescore_x = np.array(wirescore_x)

    ymin = wirescore_y.min()
    ymax = wirescore_y.max()
    xmin = wirescore_x.min()
    xmax = wirescore_x.max()
    
    fig,ax = plt.subplots(figsize=(16,8),facecolor='w')
    plt.plot(wirescore_x,wirescore_y,marker='o',linestyle='-',color='blue',markersize=10)


    minimum_v  = alg.WireBinnedScoreMinIndex(plane)
    minrange_v = alg.WireBinnedScoreMinRange(plane)
    for idx in xrange(minimum_v.size()):
        xval = wirescore_x[minimum_v[idx]]
        plt.plot([xval,xval],[0,360],marker='',linestyle='--',color='black',linewidth=2)
        xstart = wirescore_x[minrange_v[idx].first]
        xend   = wirescore_x[minrange_v[idx].second]
        ax.axvspan(xstart,xend, alpha=0.3, color='orange')

    plt.xlabel('Wire [2 wires]',fontsize=20,fontweight='bold')
    plt.ylabel('Summed Angle Difference [deg.]',fontsize=20,fontweight='bold')
    plt.tick_params(labelsize=20)
    plt.tight_layout()
    ax=plt.gca()
    ax.set_ylim(-1,ymax*1.1)
    ax.set_xlim(xmin,xmax)
    plt.grid()
    #ax.set_xlim(450,480)
    plt.show()

In [ ]:
import matplotlib.patches as patches

for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)

    alg = mgr.GetClusterAlg(3).Algo()
    
    cv=larbysimg.Manager().GetClusterAlg(3)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    
    #Getting Defect Points
    defect_data=dm.Data(1)
    compound_v = defect_data._raw_cluster_vv[plane]
    for compound in compound_v.get_cluster():
        for defect in compound.get_defects():
            pt = defect._pt_defect
            plt.plot([pt.x],[pt.y],marker='o',color='blue',markersize=10)
    
    pca_data = dm.Data(2)
    for pt in pca_data.points(plane):
        plt.plot([pt.x],[pt.y],marker='o',color='red',markersize=10)
            
    ref_data   = dm.Data(3)
    print ref_data
    plane_scan = alg.PlaneInfo(plane)

    for circle in plane_scan._circle_scan_v:
        c=patches.Circle((circle.center.x,circle.center.y),circle.radius,ec='cyan',alpha=0.05,fc='none',lw=10)
        ax.add_patch(c)
   
    if force_range:
        ymin,ymax = (yranges[plane][0],yranges[plane][1])
        xmin,xmax = (xranges[plane][0],xranges[plane][1])
    else:
        ymin,ymax = (np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        xmin,xmax = (np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    ax=plt.gca()
    #ax.set_aspect(1.0)
    #plt.savefig()
    plt.show()
    



In [ ]:
import matplotlib.patches as patches

for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    
    #for cv_ctor_lines in cv_ctor_lines_v:
    #
    #    ix+=1
    #    color=colormap[ix]
    #    for line in cv_ctor_lines:
    #        pass#plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)
    #    
    #for inter_pt in cv_inter_pts_v[plane]:
    #    pass#plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        
    alg = mgr.GetClusterAlg(3).Algo()
    
    cv=larbysimg.Manager().GetClusterAlg(3)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    
    #Getting Defect Points
    defect_data=dm.Data(1)
    compound_v = defect_data._raw_cluster_vv[plane]
    for compound in compound_v.get_cluster():
        for defect in compound.get_defects():
            pt = defect._pt_defect
            plt.plot([pt.x],[pt.y],marker='o',color='blue',markersize=10)
    
    pca_data = dm.Data(2)
    for pt in pca_data.points(plane):
        plt.plot([pt.x],[pt.y],marker='o',color='red',markersize=10)
            
    ref_data   = dm.Data(3)
    print ref_data
    plane_scan = alg.PlaneInfo(plane)

    for circle in plane_scan._circle_scan_v:
        c=patches.Circle((circle.center.x,circle.center.y),circle.radius,ec='cyan',alpha=0.05,fc='none',lw=10)
        ax.add_patch(c)
        
    minimum_v = alg.TimeBinnedScoreMinIndex()
    for idx in xrange(minimum_v.size()):
        xval = minimum_v[idx] + alg.TimeBinMin()
        plt.plot([xval,xval],[0,2000],marker='',linestyle='--',color='magenta',linewidth=2)

    vtx_vv = ref_data.get_circle_vertex()
    for vtx_idx in xrange(len(vtx_vv)):
        vtx = vtx_vv[vtx_idx][plane]
        color='magenta'
        if ref_data.get_type(vtx_idx) == 1:
            color='yellow'
        plt.plot([vtx.center.x],[vtx.center.y],marker='$\star$',color=color,markersize=24)

    if force_range:
        ymin,ymax = (yranges[plane][0],yranges[plane][1])
        xmin,xmax = (xranges[plane][0],xranges[plane][1])
    else:
        ymin,ymax = (np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        xmin,xmax = (np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)    

    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    ax=plt.gca()
    #ax.set_aspect(1.0)
    #plt.savefig()
    plt.show()
    



In [ ]:
mgr=larbysimg.Manager()
dm=mgr.DataManager()

data=dm.Data(3)
for plane in xrange(3):
    print 'Plane',plane
    xmin,xmax = (1e10,0)
    ymin,ymax = (1e10,0)
    vtx2d_x = []
    vtx2d_y = []
    radius  = []
    xs_pt_x = []
    xs_pt_y = []
    for vtx_idx in xrange(data.num_vertex()):
        vtx3d = data.get_vertex(vtx_idx)
        vtx2d = data.get_circle_vertex(vtx_idx,plane)
        print 'Vertex Index',vtx_idx,
        print '@ 2D pos (',int(vtx2d.center.x+0.5),',',int(vtx2d.center.y+0.5),')',
        print '# crossing',vtx2d.xs_v.size()
        vtx2d_x.append(vtx2d.center.x)
        vtx2d_y.append(vtx2d.center.y)
        radius.append(vtx2d.radius)
        for xs in vtx2d.xs_v:
            xs_pt_x.append(xs.pt.x)
            xs_pt_y.append(xs.pt.y)

    xmin,xmax = (1e10,0)
    ymin,ymax = (1e10,0)
    for x in vtx2d_x + xs_pt_x:
        if xmin > x : xmin = x
        if xmax < x : xmax = x
    for y in vtx2d_y + xs_pt_y:
        if ymin > y : ymin = y
        if ymax < y : ymax = y
    xmin -= 20
    xmax += 20
    ymin -= 20
    ymax += 20
    
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    #Get the image
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)

    for i in xrange(len(vtx2d_x)):
        c=patches.Circle((vtx2d_x[i],vtx2d_y[i]),radius[i],ec='cyan',fc='none',lw=5)
        ax.add_patch(c)
        
    for i in xrange(len(vtx2d_x)):
        plt.plot([vtx2d_x[i]],[vtx2d_y[i]],marker='o',color='blue',markersize=10)
        
    for i in xrange(len(xs_pt_x)):
        plt.plot([xs_pt_x[i]],[xs_pt_y[i]],marker='o',color='magenta',markersize=8)

    if force_range:
        ax.set_ylim(yranges[plane][0],yranges[plane][1])
        ax.set_xlim(xranges[plane][0],xranges[plane][1])
    else:
        plt.xlim(xmin,xmax)
        plt.ylim(ymin,ymax)

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    ax=plt.gca()
    #ax.set_aspect(1.0)
    #plt.savefig()
    plt.show()
    



In [ ]:
#New VertexCluster

data=dm.Data(4)
#Simply a list of VertexCluster (i.e. vertex-wise list of clusters)
all_vtx_cluster=data._vtx_cluster_v
print "There are ",len(all_vtx_cluster),"vertexs found in this event. Note that this is vertex-wise"
print
#print all_vtx_cluster
################
target_vertex   = None
target_plane    = None
target_compound = None
for vtx_idx in xrange(len(all_vtx_cluster)):
        
    vtx_particles = all_vtx_cluster[vtx_idx]
    if target_vertex is not None and not target_vertex == vtx_particles.id(): continue
    for plane in xrange(3):
        if target_plane is not None and not target_plane == plane: continue
        vtx2d = dm.Data(3).get_circle_vertex(vtx_particles.id())[plane]
        xmin,xmax = (vtx2d.center.x-200,vtx2d.center.x+200)
        ymin,ymax = (vtx2d.center.y-200,vtx2d.center.y+200)
        
        vtx_clusters =  all_vtx_cluster[vtx_idx]
        clusters     =  vtx_clusters.get_clusters(plane)
        vtx_3d       =  vtx_clusters.get_vertex()
        vtx_2d       =  vtx_3d.vtx2d_v
        vtx_2d_plane =  vtx_2d[plane]
        
        print 'Vertex ID',vtx_clusters.id(),'plane',plane,
        print '... contains',vtx_clusters.num_pixels(plane),'non-zero pixels'
        
        #Get the image
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img = img_v[plane]
        shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
        #shape_img[:,::-1]=shape_img
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)
            
        circles       =  all_vtx_cluster[vtx_idx].get_circle_vertex(plane)
        center        =  circles.center
        num_pixel_sum = 0
        for c_idx in xrange(clusters.size()):
            cluster = clusters[c_idx]
            if target_compound is not None and not target_compound == c_idx: continue
            print 'Cluster ID',c_idx,'ctor size',cluster._ctor.size(),
            print '# non-zero pixels',cluster._num_pixel,
            print '(fraction',vtx_clusters.num_pixel_fraction(plane,c_idx),')'
            num_pixel_sum = cluster._num_pixel
            ctor=cluster._ctor
            pts=[[pt.x,pt.y] for pt in ctor]
            pts.append(pts[0])
            pts=np.array(pts)
            plt.plot(pts[:,0],pts[:,1],'-o')
            plt.plot(center.x, center.y, marker='$\star$',color='pink',markersize=30)
            
            
        print 'Total pixel fraction:',vtx_clusters.num_pixel_fraction(plane)

        if force_range:
            ax.set_ylim(yranges[plane][0],yranges[plane][1])
            ax.set_xlim(xranges[plane][0],xranges[plane][1])
        else:
            plt.xlim(xmin,xmax)
            plt.ylim(ymin,ymax)

        plt.xlabel('Time [6 ticks]',fontsize=20)
        plt.ylabel('Wire [2 wires]',fontsize=20)
        plt.tick_params(labelsize=20)
        ax.set_aspect(0.8)
        plt.tight_layout()
        ax=plt.gca()
        
        #plt.ylim(920,955)
        #plt.xlim(625,660)
        plt.show()

In [ ]:
mgr=larbysimg.Manager()
dm=mgr.DataManager()
target_vtx      = None
target_plane    = None
target_compound = None
data=dm.Data(5)
for vtx_particles in data.get_vertex_clusters():
    if target_vtx is not None and not vtx_particles.id() == target_vtx: continue
    for plane in xrange(3):
        if target_plane is not None and not target_plane == plane: continue
        print 'Vertex ID',vtx_particles.id(),'plane',plane

        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
        #Get the image
        shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
        #shape_img[:,::-1]=shape_img
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)
        
        xmin,xmax = (1e9,0)
        ymin,ymax = (1e9,0)
        
        for compound in vtx_particles.get_cluster(plane):
            if target_compound is not None and not target_compound == compound.id(): continue
            print 'Particle',compound.id(),'has',compound.get_atoms().size(),'atoms'
            cid=0
            for atom in compound.get_atoms():
                print '  Atom',atom.id(),'size',atom._ctor.size(),
                print 'associated with',atom.associated_defects().size(),'defects',
                print 'color=%s' % colormap[cid%len(colormap)]
                for defect_id in atom.associated_defects():
                    defect_pt = compound.get_defect(defect_id)._pt_defect
                    #print '    defect',defect_id,'(%d,%d)' % (int(defect_pt.x),int(defect_pt.y))
                    plt.plot([defect_pt.x],[defect_pt.y],marker='$\star$',
                             color=colormap[cid%len(colormap)],markersize=32, alpha=0.5)
                cid += 1
            cid=0
            for atom in compound.get_atoms():
                pts = [[pt.x,pt.y] for pt in atom._ctor]
                pts.append(pts[0])
                pts=np.array(pts)
                plt.plot(pts[:,0],pts[:,1],'-o',color=colormap[cid%len(colormap)])
                cid+=1
                if xmin > pts[:,0].min(): xmin = pts[:,0].min()
                if xmax < pts[:,0].max(): xmax = pts[:,0].max()
                if ymin > pts[:,1].min(): ymin = pts[:,1].min()
                if ymax < pts[:,1].max(): ymax = pts[:,1].max()
                    
            for defect in compound.get_defects():
                ax.plot([0,900],[defect._split_line.y(0),defect._split_line.y(900)],color='black')

        ax.set_aspect(1.0)
        
        if force_range:
            ax.set_ylim(yranges[plane][0],yranges[plane][1])
            ax.set_xlim(xranges[plane][0],xranges[plane][1])
        else:
            plt.xlim(xmin-40,xmax+40)
            plt.ylim(ymin-40,ymax+40)

        plt.xlabel('Time [6 ticks]',fontsize=20)
        plt.ylabel('Wire [2 wires]',fontsize=20)
        plt.tick_params(labelsize=20)
        ax.set_aspect(0.8)
        plt.tight_layout()
        ax=plt.gca()
        
        #ax.set_ylim(920,955)
        #ax.set_xlim(625,660)
        plt.show()

In [ ]:
mgr=larbysimg.Manager()
dm=mgr.DataManager()
data=dm.Data(7)
target_vertex   = None
target_plane    = None
target_compound = None
for vtx_dqdx in data.get_vertex_clusters():
    if target_vertex is not None and not target_vertex == vtx_dqdx.id(): continue
    for plane in xrange(3):
        if target_plane is not None and not target_plane == plane: continue
        print 'Vertex ID',vtx_dqdx.id(),'plane',plane
        
        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
        #Get the image
        shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
        #shape_img[:,::-1]=shape_img
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        vtx_particles = dm.Data(5).get_vertex_cluster(vtx_dqdx.id())
        
        xmin,xmax=(1e9,0)
        ymin,ymax=(1e9,0)
        
        for compound in vtx_particles.get_cluster(plane):
            if target_compound is not None and not target_compound == compound.id(): continue
            part_dqdx = vtx_dqdx.get_cluster(plane)[compound.id()]
            print 'Particle ID',part_dqdx.id(),compound.id()
            
            for atom_idx in xrange(part_dqdx.num_atoms()):
                atom_start = part_dqdx.atom_start_pt(atom_idx)
                atom_end   = part_dqdx.atom_end_pt(atom_idx)
                atom_id    = part_dqdx.atom_id(atom_idx)
                atom_ctor  = compound.get_atom(atom_id)._ctor

                print 'atom id:',atom_id,
                print '# defects=%d' % compound.get_atom(atom_id).associated_defects().size(),
                print 'start @ (%d,%d)' % (int(atom_start.x),int(atom_start.y)),
                print 'end @ (%d,%d)' % (int(atom_end.x),int(atom_end.y)),
                print 'color:', colormap[atom_idx%len(colormap)]
                
                pts = np.array([[pt.x,pt.y] for pt in atom_ctor])
                plt.plot(pts[:,0],pts[:,1],'-o',color=colormap[atom_idx%len(colormap)])
                
                if xmin > pts[:,0].min(): xmin = pts[:,0].min()
                if xmax < pts[:,0].max(): xmax = pts[:,0].max()
                if ymin > pts[:,1].min(): ymin = pts[:,1].min()
                if ymax < pts[:,1].max(): ymax = pts[:,1].max()
                
                plt.plot([atom_start.x],[atom_start.y],marker='$\star$',
                         color=colormap[atom_idx%len(colormap)],markersize=32, alpha=0.4)
                plt.plot([atom_end.x],[atom_end.y],marker='$\\times$',
                         color=colormap[atom_idx%len(colormap)],markersize=32, alpha=0.4)
                
        if force_range:
            ax.set_ylim(yranges[plane][0],yranges[plane][1])
            ax.set_xlim(xranges[plane][0],xranges[plane][1])
        else:
            plt.xlim(xmin-40,xmax+40)
            plt.ylim(ymin-40,ymax+40)

        plt.xlabel('Time [6 ticks]',fontsize=20)
        plt.ylabel('Wire [2 wires]',fontsize=20)
        plt.tick_params(labelsize=20)
        ax.set_aspect(0.8)
        plt.tight_layout()
        ax=plt.gca()
        plt.show()
        
        for part_dqdx in vtx_dqdx.get_cluster(plane):   
            if target_compound is not None and not target_compound == part_dqdx.id(): continue
            dqdx = part_dqdx.dqdx()
            start_index_v = part_dqdx.atom_start_index_array()
            range_v = []
            for i in xrange(start_index_v.size()):
                if not i: continue
                range_v.append((start_index_v[i-1],start_index_v[i]))                
            range_v.append((start_index_v.back(),dqdx.size()-1))
            if not(range_v): continue
                
            fig,ax = plt.subplots(figsize=(12,6),facecolor='w')
            for i in xrange(len(range_v)):
                start,end = range_v[i]
                data_y = []
                data_x = []
                while 1:
                    data_y.append(dqdx[start])
                    data_x.append(start)
                    start +=1
                    if start == end: break
                print range_v[i],'/',dqdx.size(),'...',
                spt2d = part_dqdx.atom_start_pt(i)
                ept2d = part_dqdx.atom_end_pt(i)
                print '(%d,%d) => (%d,%d)' % (int(spt2d.x),int(spt2d.y),int(ept2d.x),int(ept2d.y))
                plt.plot(data_x,data_y,marker='o',color=colormap[i%len(colormap)])
                
            plt.xlabel('Pixel Distance',fontsize=20)
            plt.ylabel('Pixel Intensity',fontsize=20)
            plt.tick_params(labelsize=20)
            plt.grid()
            plt.tight_layout()
            ax=plt.gca()
            plt.show()
            print 'particle',part_dqdx.id(),'done'